# Part 2: Training

In this part we will show how using MLRun's **Feature Store** we can easily define a **Feature Vector** and create the dataset we need to run our training process.  
By the end of this tutorial you’ll learn how to:
- Combine multiple data sources to a single Feature Vector
- Create training dataset
- Create a model using an MLRun Hub function

In [1]:
project_name = 'fraud-demo'

In [2]:
import mlrun

# Initialize the MLRun project object
project = mlrun.get_or_create_project(project_name, context="./", user_project=True)

> 2021-11-09 15:40:13,556 [info] loaded project fraud-demo from MLRun DB


## Step 1 - Create a Feature Vector  
In this section we will create our Feature Vector.  
The Feature vector will have a `name` so we can reference to it later via the URI or our serving function, and a list of `features` from the available FeatureSets.  We can add a feature from a feature set by adding `<FeatureSet>.<Feature>` to the list, or add `<FeatureSet>.*` to add all the FeatureSet's available features.  

By default, the first FeatureSet in the feature list will act as the spine. meaning that all the other features will be joined to it.  
For example, in this instance we use the early_sense sensor data as our spine, so for each early_sense event we will create produce a row in the resulted Feature Vector.

In [3]:
# Define the list of features we will be using
features = ['transactions.amount_max_2h', 
            'transactions.amount_sum_2h', 
            'transactions.amount_count_2h',
            'transactions.amount_avg_2h', 
            'transactions.amount_max_12h', 
            'transactions.amount_sum_12h',
            'transactions.amount_count_12h', 
            'transactions.amount_avg_12h', 
            'transactions.amount_max_24h',
            'transactions.amount_sum_24h', 
            'transactions.amount_count_24h', 
            'transactions.amount_avg_24h',
            'transactions.es_transportation_count_14d', 
            'transactions.es_health_count_14d',
            'transactions.es_otherservices_count_14d', 
            'transactions.es_food_count_14d',
            'transactions.es_hotelservices_count_14d', 
            'transactions.es_barsandrestaurants_count_14d',
            'transactions.es_tech_count_14d', 
            'transactions.es_sportsandtoys_count_14d',
            'transactions.es_wellnessandbeauty_count_14d', 
            'transactions.es_hyper_count_14d',
            'transactions.es_fashion_count_14d', 
            'transactions.es_home_count_14d', 
            'transactions.es_travel_count_14d', 
            'transactions.es_leisure_count_14d',
            'transactions.gender_F',
            'transactions.gender_M',
            'transactions.step', 
            'transactions.amount', 
            'transactions.timestamp_hour',
            'transactions.timestamp_day_of_week',
            'events.*']

In [4]:
# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the feature vector name for future reference
fv_name = 'transactions-fraud'

# Define the feature vector using our Feature Store (fstore)
transactions_fv = fstore.FeatureVector(fv_name, 
                          features, 
                          label_feature="labels.label",
                          description='Predicting a fraudulent transaction')

# Save the feature vector in the Feature Store
transactions_fv.save()

## Step 2 - Preview the Feature Vector Data

Obtain the values of the features in the feature vector, to ensure the data appears as expected

In [5]:
# Import the Parquet Target so we can directly save our dataset as a file
from mlrun.datastore.targets import ParquetTarget

# Get offline feature vector as dataframe and save the dataset to parquet
train_dataset = fstore.get_offline_features(fv_name, target=ParquetTarget())

> 2021-11-09 15:40:14,914 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-admin/FeatureStore/transactions-fraud/parquet/vectors/transactions-fraud-latest.parquet', 'status': 'ready', 'updated': '2021-11-09T15:40:14.914186+00:00', 'size': 1934256}


In [6]:
# Preview our dataset
train_dataset.to_dataframe().tail(5)

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,amount_sum_12h,amount_count_12h,amount_avg_12h,amount_max_24h,amount_sum_24h,...,gender_F,gender_M,step,amount,timestamp_hour,timestamp_day_of_week,event_details_change,event_login,event_password_change,label
49996,2.95,2.95,1.0,2.95,30.69,42.03,3.0,14.01,50.23,92.26,...,1,0,41,2.95,14,1,0.0,0.0,1.0,0
49997,20.31,20.31,1.0,20.31,20.31,20.31,1.0,20.31,20.31,20.31,...,0,1,119,20.31,14,1,0.0,1.0,0.0,0
49998,24.87,22.69,0.0,NaN,24.87,27.00,2.0,13.50,39.73,66.73,...,0,1,26,24.87,14,1,0.0,1.0,0.0,0
49999,9.75,9.75,1.0,9.75,9.75,9.75,1.0,9.75,9.75,14.65,...,1,0,124,9.75,14,1,0.0,0.0,1.0,0
50000,32.51,32.51,1.0,32.51,32.51,116.80,5.0,23.36,51.18,278.37,...,0,1,24,32.51,14,1,0.0,0.0,1.0,0


## Step 3 - Train Models and Choose Highest Accuracy

With MLRun, one can easily train different models and compare the results. In the code below, we train 3 different models,
each uses a different algorithm (random forest, XGBoost, adabost), and choose the model with the highest accuracy

In [7]:
# Import the Sklearn classifier function from the functions hub
classifier_fn = mlrun.import_function('hub://sklearn-classifier')

In [8]:
# Prepare the parameters list for the training function
# We will be using 3 different models
training_params = {"model_name": ['transaction_fraud_rf', 
                                  'transaction_fraud_xgboost', 
                                  'transaction_fraud_adaboost'],
              
              "model_pkg_class": ['sklearn.ensemble.RandomForestClassifier',
                                  'sklearn.ensemble.GradientBoostingClassifier',
                                  'sklearn.ensemble.AdaBoostClassifier']}

# Define the training task, including our feature vector, label and hyperparams definitions
train_task = mlrun.new_task('training', 
                      inputs={'dataset': transactions_fv.uri},
                      params={'label_column': 'label'}
                     )

train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

# Specify our cluster image
classifier_fn.spec.image = 'mlrun/mlrun'

# Run training
classifier_fn.run(train_task, local=False)

> 2021-11-09 15:40:15,423 [info] starting run training uid=cbd51dc452ed4170b04f0f30a937be51 DB=http://mlrun-api:8080
> 2021-11-09 15:40:15,660 [info] Job is running in the background, pod: training-gk6f4
> 2021-11-09 15:40:34,554 [info] best iteration=3, used criteria max.accuracy
> 2021-11-09 15:40:35,350 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-admin,...a937be51,0,Nov 09 15:40:18,completed,training,v3io_user=adminkind=jobowner=admin,dataset,label_column=label,best_iteration=3accuracy=0.9928057553956835test-error=0.007194244604316547rocauc=0.9361692796842451brier_score=0.20558286473191562f1-score=0.6833333333333333precision_score=0.82recall_score=0.5857142857142857,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_results


> 2021-11-09 15:40:38,077 [info] run executed, status=completed


## Step 4 - Perform Feature Selection

As part of our data science process we will try and reduce the training dataset's size to get rid of bad or unuseful features and save computation time.

We will use our ready-made feature selection function from our hub [`hub://feature_selection`](https://github.com/mlrun/functions/blob/development/feature_selection/feature_selection.ipynb) to select the best features to keep on a sample from our dataset and run the function on that.


In [9]:
feature_selection_fn = mlrun.import_function('hub://feature_selection')

feature_selection_run = feature_selection_fn.run(
            params={'sample_ratio':0.25,
                    'output_vector_name':fv_name + "-short",
                   'ignore_type_errors': True},
    
            inputs={'df_artifact': transactions_fv.uri},
            name='feature_extraction',
            handler='feature_selection',
    local=False)

> 2021-11-09 15:40:38,393 [info] starting run feature_extraction uid=8effa3f230564601b2f83106cff99afa DB=http://mlrun-api:8080
> 2021-11-09 15:40:38,621 [info] Job is running in the background, pod: feature-extraction-njwbx
> 2021-11-09 15:40:45,600 [info] Couldn't calculate chi2 because of: Input X must be non-negative.
> 2021-11-09 15:40:50,196 [info] votes needed to be selected: 3
> 2021-11-09 15:40:51,380 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-admin/FeatureStore/transactions-fraud-short/parquet/vectors/transactions-fraud-short-latest.parquet', 'status': 'ready', 'updated': '2021-11-09T15:40:51.380829+00:00', 'size': 673253}
> 2021-11-09 15:40:51,555 [info] run executed, status=completed
Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.
final 

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-admin,...cff99afa,0,Nov 09 15:40:42,completed,feature_extraction,v3io_user=adminkind=jobowner=adminhost=feature-extraction-njwbx,df_artifact,sample_ratio=0.25output_vector_name=transactions-fraud-shortignore_type_errors=True,top_features_vector=store://feature-vectors/fraud-demo-admin/transactions-fraud-short,f_classifmutual_info_classiff_regressionLinearSVCLogisticRegressionExtraTreesClassifierfeature_scoresmax_scaled_scores_feature_scoresselected_features_countselected_features


> 2021-11-09 15:40:54,048 [info] run executed, status=completed


In [10]:
mlrun.get_dataitem(feature_selection_run.outputs['top_features_vector']).as_df().tail(5)

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,label
49996,2.95,2.95,1.0,2.95,30.69,0
49997,20.31,20.31,1.0,20.31,20.31,0
49998,24.87,22.69,0.0,NaN,24.87,0
49999,9.75,9.75,1.0,9.75,9.75,0
50000,32.51,32.51,1.0,32.51,32.51,0


## Step 5 - Train our models with top features

Following the feature selection, we train new models using the resultant features. We can observe the accuracy and other results remain high
meaning we get a model that requires less features to be accurate and thus less error-prone.

In [11]:
# Defining our training task, including our feature vector, label and hyperparams definitions
ensemble_train_task = mlrun.new_task('training', 
                      inputs={'dataset': feature_selection_run.outputs['top_features_vector']},
                      params={'label_column': 'label'}
                     )
ensemble_train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

classifier_fn.run(ensemble_train_task)

> 2021-11-09 15:40:54,152 [info] starting run training uid=af70b83e3c604e51963044a665231604 DB=http://mlrun-api:8080
> 2021-11-09 15:40:54,382 [info] Job is running in the background, pod: training-kh2dx
> 2021-11-09 15:41:06,491 [info] best iteration=3, used criteria max.accuracy
> 2021-11-09 15:41:07,524 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-admin,...65231604,0,Nov 09 15:40:57,completed,training,v3io_user=adminkind=jobowner=admin,dataset,label_column=label,best_iteration=3accuracy=0.9912280701754386test-error=0.008771929824561403rocauc=0.920616310497263brier_score=0.1796708804829256f1-score=0.584070796460177precision_score=0.7021276595744681recall_score=0.5,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_results


> 2021-11-09 15:41:13,719 [info] run executed, status=completed


## Done!

You've completed Part 2 of the model training with the feature store.
Proceed to [Part 3](03-deploy-serving-model.ipynb) to learn how to deploy and monitor the model.